In [1]:
from RusPhonetic import phonetic_module
import string
from ru_accent_poet import accent_line
import re
import csv
import pandas as pd
import os
import sqlite3

Функции, подготавливающие текст к фонемному разбору (*has_cyrillic, has_latin, has_number, text_cleaning*).

In [2]:
def has_cyrillic(text):
    return bool(re.search("[а-яА-Я]", text))

In [3]:
def has_latin(text):
    return bool(re.search("[a-zA-Z]", text))

In [4]:
def has_number(text):
    return bool(re.search("[0-9]", text))

In [6]:
def text_cleaning(text):
    punct = (string.punctuation + '«»' + '…').replace("'", '')
    text = [i.strip(punct).replace('-', '') for i in text.lower().split()]
    text = [i for i in text if not has_latin(i) and not has_number(i) and has_cyrillic(i)]
    return text

Функция, которая предназначена для исправления некоторых фонемных разборов, полученных RusPhonetic.

In [7]:
def replace_u(word):
    vowels = "аяоеуюыиэё"
    if word.count('ю') == 1:
        if 'ю' != word[0] and 'ью' not in word and word[word.index('ю')-1] not in vowels:
            new_word = word.replace('ю', 'ью')
            return new_word, 1
        else:
            return word, 0
    else:
        return word, 0

Функция определяет номер ударного слога.

In [8]:
def stress_position(stressed_word):  # требуется слово с размеченным ударением
    vowels = "аяоеуюыиэё"
    word_vowels = []
    flag = 0
    for letter in stressed_word:
        flag += 1
        if letter in vowels:
            if flag != len(stressed_word):
                if stressed_word[flag] == "'" or stressed_word[flag] == " ":
                    word_vowels.extend([letter, "+"])
                else:
                    word_vowels.append(letter)
            else:
                word_vowels.append(letter)
    if len(word_vowels) > 1:
        if "ё" not in stressed_word:
            try:
                stressed_position = word_vowels.index("+")
            except ValueError:
                return None
        else:
            stressed_position = word_vowels.index("ё")+1
    else:
        stressed_position = 1
    return stressed_position

Функция осуществляет фонемный разбор текста на основе модулей *ru_accent_poet* (расставляет ударение) и *RusPhonetic* (проводит фонемный разбор слова).

In [9]:
def text_transcription(text):
    text_cleaned = text_cleaning(text)
    accented_text = text_cleaning(accent_line(text))
    text_transc = []
    vowels = "аоуыиэ"
    for word in text_cleaned:
        if stress_position(accented_text[text_cleaned.index(word)]) is not None:
            st_position = stress_position(accented_text[text_cleaned.index(word)])
            word_transc = phonetic_module.Phonetic(replace_u(word)[0], st_position).get_phonetic()
            if replace_u(word)[1] == 1:
                word_transc = word_transc.replace("й'у", "у")
            flag_vow = 0
            flag_ph = 0
            for phonem in word_transc:
                flag_ph += 1
                if phonem in vowels:
                    flag_vow += 1
                    if flag_vow == st_position:
                        text_transc.append(word_transc[:flag_ph] + '+' + word_transc[flag_ph:])
    return text_transc

Функции, собирающие данные для проверки гипотезы H0 с помощью критерия хи-квадрат.

In [10]:
def get_phonems(text_transcripted):
    text_transcripted = [i.replace("'", '').replace(':', '') for i in text_transcripted]
    return list(set(''.join(text_transcripted)))

In [11]:
def phonem_sep(lemma):  # работает с одним словом
    st = lemma.index('+')
    phonems = list(lemma[:st-1]) + [lemma[st-1:st+1]] + list(lemma[st+1:])
    phonems = [i for i in phonems if i != ':' and i != "'"]
    return phonems

In [12]:
# text_transcripted - список из слов с выполненным фонетическим разбором
def count_phonems(text_transcripted):
    phonems = 0
    for i in text_transcripted:
        phonems += len(phonem_sep(i))
    return phonems

In [13]:
# считается встречаемость одной фонемы
def count_phonem_1(ph, text_transcripted):
    return ''.join(text_transcripted).count(ph)

Распределение частот фонем в речи, эмпирически найденных Генри Кучера в современной русской прозе.

(H. Kučera. Entropy, redundancy and functional load in Russian and Czech // *American Contr. to the 5th International Congress of Slavists* 1. The Hague: Mouton, 1963. P. 191–218.)

В файле *ph_kuc_baev.csv* находятся фонемы Кучера, записанные латиницей, и соответсвующие им фонемы, записанные кириллицей (знаком "+" обозначено ударение, значения парных и твердых согласных объединялись).

In [14]:
with open('ph_kuc_baev.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    phons_dct = {}
    for row in reader:
        phons_dct[row['ph_cyr']] = {'freq': round(float(row['freq'])/1000, 6), 'ph_lat': row['ph_lat']}

In [15]:
data = pd.read_csv('ph_kuc_baev.csv', delimiter=';')
data = data.set_index('ph_lat')
data = data.fillna('None')
data = pd.DataFrame(data)

In [16]:
data

,freq,ph_cyr
ph_lat,,
i,116.857,и
í,29.146,и+
u,22.512,у
ú,14.508,у+
a,132.119,а
á,48.561,а+
ó,41.909,о+
é,27.314,э+
j,42.858,й


In [101]:
def get_frequency(ph):
    if ph == 'ы':
        return phons_dct['и']['freq']
    elif ph == 'ы+':
        return phons_dct['и+']['freq']
    elif ph == 'э':
        return phons_dct['и']['freq']
    elif ph == 'щ':
        return phons_dct['ш']['freq']
    elif ph == 'о':
        return phons_dct['о+']['freq']
    else:
        return phons_dct[ph]['freq']

Функция, проверяющая гипотезу H0 (частота фонемы в тексте не отличается от ее частоты в речи) на нескольких уровнях значимости (0.1: 2.71; 0.05: 3.84; 0.01: 6.64).

In [102]:
def hypothesis_h0(lemma, text_transcripted):
    phonems = phonem_sep(lemma)
    lemma_data = dict.fromkeys(phonems, True)
    all_ph = count_phonems(text_transcripted)
    for phon in lemma_data.keys():
        ph1_total = count_phonem_1(phon, text_transcripted)
        if ph1_total != 0:
            frequency = get_frequency(phon)
            if all_ph * frequency >= 4:
                criteria_x = (ph1_total - all_ph * frequency)**2 / (all_ph * frequency * (1 - frequency))
                if criteria_x > 6.64:  # 2.71 3.84 6.64
                    lemma_data[phon] = False
            else:
                all_ph1 = all_ph
                while all_ph1 * frequency < 4:
                    all_ph1 += 1
                criteria_x = (ph1_total - all_ph1 * frequency)**2 / (all_ph1 * frequency * (1 - frequency))
                if criteria_x > 6.64:  # 2.71 3.84 6.64
                    lemma_data[phon] = False
        else:
            lemma_data[phon] = None
    return lemma_data

Функция собирает полные анаграммы.

In [103]:
def word_collector(lemma_data): # все подходящие слова
    if None not in lemma_data.values():
        if len(lemma_data) - list(lemma_data.values()).count(False) == 0: # !!!
            return lemma_data

Функции, собирающие неполные анаграммы 3 типов (*word_collector2, word_collector3, word_collector4*).

In [104]:
def word_collector2(lemma_data): # все подходящие слова без учета гласных
    vowels = ['а', 'о', 'у', 'ы', 'и', 'э', 'а+', 'о+', 'у+', 'ы+', 'и+', 'э+']
    cons_lst = [i for i in list(lemma_data) if i not in vowels]
    cons_counter = 0
    if None not in lemma_data.values():
        for i in cons_lst:
            if lemma_data[i] == False:
                cons_counter += 1
        if len(cons_lst) - cons_counter == 0:  # !!!
            return lemma_data

In [105]:
def word_collector3(lemma_data): # все подходящие слова без учета гласных и без 1 согласной
    vowels = ['а', 'о', 'у', 'ы', 'и', 'э', 'а+', 'о+', 'у+', 'ы+', 'и+', 'э+']
    cons_lst = [i for i in list(lemma_data) if i not in vowels]
    cons_counter = 0
    if None not in lemma_data.values():
        for i in cons_lst:
            if lemma_data[i] == False:
                cons_counter += 1
        if len(cons_lst) - cons_counter == 1:  # !!!
            return lemma_data

In [106]:
def word_collector4(lemma_data): # все подходящие слова без 1 фонемы
    if None not in lemma_data.values():
        if len(lemma_data) - list(lemma_data.values()).count(False) == 1: # !!!
            return lemma_data

Функция, анализируя фонетически разобранное стихотворение, собирает анаграммы 4 типов с помощью критерия хи-квадрат и базы данных, содержащей леммы из МАС и их фонетический разбор. 

In [18]:
con = sqlite3.connect('data.db')
cur = con.cursor()
lem_query = """
SELECT DISTINCT word_nom, transcripted FROM dictionary
"""
cur.execute(lem_query)
result = cur.fetchall()
print(len(result))

77195


In [19]:
result[10]

('вагонетчица', "ваган'э+ч':ица")

In [107]:
# собирает анаграммы одного фонетически разобранного стихотворения
def anagrams(poem_tr):
    a_all = []  # полные анаграммы (учитываются все фонемы слова)
    a_cons = []  # анаграммы без учета гласных
    a_cons1 = []  # анаграммы без учета гласных и одной из согласных
    a_all1 = []  # анаграммы без учета одной фонемы
    for pair in result:  # берется из базы данных data.db
        lemma_data = hypothesis_h0(pair[1], poem_tr)
        a = word_collector(lemma_data)
        b = word_collector2(lemma_data)
        c = word_collector3(lemma_data)
        d = word_collector4(lemma_data)
        if a:
            a_all.append(pair)
        if b:
            a_cons.append(pair)
        if c:
            a_cons1.append(pair)
        if d:
            a_all1.append(pair)
    return a_all, a_cons, a_cons1, a_all1

*ph_to_lett, data_to_base* - сбор данных в удобном для записи в базу формате.

In [108]:
def ph_to_lett(anagr_tpl):
    t1 = []
    t2 = []
    t3 = []
    t4 = []
    for i in anagr_tpl[0]:
        t1.append(i[0])
    for i in anagr_tpl[1]:
        t2.append(i[0])
    for i in anagr_tpl[2]:
        t3.append(i[0])
    for i in anagr_tpl[3]:
        t4.append(i[0])
    return t1, t2, t3, t4

In [78]:
data_to_base = []
for i in respo:
    indx = i[0]  # индекс
    txtau = i[1]  # автор
    txtp = i[2]  # текст
    tr = i[3].split()  # транскрипция текста - список
    txttr = i[3]  # транскрипция текста - строка
    alles = ph_to_lett(anagrams(tr))  # все анаграммы
    all_true = '\n'.join(alles[0]) # полная анаграмма
    all_cons = '\n'.join(alles[1]) # только согласные
    cons_1 = '\n'.join(alles[2]) # только согласные без одного
    all_1 = '\n'.join(alles[3]) # все фонемы кроме одной
    data_to_base.append((indx, txtau, txttr, all_true, len(alles[0]), all_cons, len(alles[1]), cons_1, len(alles[2]), all_1, len(alles[3])))